# Question 3

## Ingesting the dataset into a SQL db

In [1]:
# Import modules
import sqlite3
import pandas as pd

# Connecting to the db and getting a curson to execute queries
conn = sqlite3.connect('Chinook_Sqlite.sqlite')
cursor = conn.cursor()

# checking a few rows
cursor.execute("SELECT * FROM Album")
rows = cursor.fetchall()

for r in range(10):
    print(rows[r])

(1, 'For Those About To Rock We Salute You', 1)
(2, 'Balls to the Wall', 2)
(3, 'Restless and Wild', 2)
(4, 'Let There Be Rock', 1)
(5, 'Big Ones', 3)
(6, 'Jagged Little Pill', 4)
(7, 'Facelift', 5)
(8, 'Warner 25 Anos', 6)
(9, 'Plays Metallica By Four Cellos', 7)
(10, 'Audioslave', 8)


## Query 1: Find the top 5 most popular genres by total sales.

In [69]:
# Getting the data 
cursor.execute("""
    select g.Name as genreName, sum(Quantity*i.UnitPrice) as totalSales
    from Genre g
    inner join Track t
        on g.GenreId = t.GenreId
    inner join InvoiceLine i
        on t.TrackId = i.TrackId
    group by 1
    order by 2 desc
    limit 5
""")
rows = cursor.fetchall()

# printing them
rows

[('Rock', 826.65),
 ('Latin', 382.14),
 ('Metal', 261.36),
 ('Alternative & Punk', 241.56),
 ('TV Shows', 93.53)]

In [71]:
# exporting the results to csv
colnames = [d[0] for d in cursor.description]
pd.DataFrame(rows).to_csv('Question3_Task2_Query1.csv', header=colnames, index=False)

## Query 2: Calculate the average invoice total by country.

In [65]:
# Getting the data 
cursor.execute(""" 
    select BillingCountry, avg(total) as avgInvoiceTotal
    from Invoice
    group by 1
    order by 2 desc
""")
rows = cursor.fetchall()

# and checking a few rows
rows

[('Chile', 6.659999999999999),
 ('Ireland', 6.517142857142857),
 ('Hungary', 6.517142857142857),
 ('Czech Republic', 6.445714285714286),
 ('Austria', 6.088571428571428),
 ('Finland', 5.945714285714286),
 ('Netherlands', 5.8028571428571425),
 ('India', 5.78923076923077),
 ('USA', 5.747912087912088),
 ('Norway', 5.659999999999999),
 ('Germany', 5.588571428571428),
 ('France', 5.574285714285714),
 ('Sweden', 5.517142857142857),
 ('Portugal', 5.517142857142857),
 ('Brazil', 5.4314285714285715),
 ('Canada', 5.4278571428571425),
 ('United Kingdom', 5.3742857142857146),
 ('Spain', 5.374285714285714),
 ('Poland', 5.374285714285714),
 ('Italy', 5.374285714285714),
 ('Denmark', 5.374285714285714),
 ('Belgium', 5.374285714285714),
 ('Australia', 5.374285714285714),
 ('Argentina', 5.374285714285714)]

In [67]:
# exporting the results to csv
colnames = [d[0] for d in cursor.description]
pd.DataFrame(rows).to_csv('Question3_Task2_Query2.csv', header=colnames, index=False)

## Query 3: Identify the top 3 most valued customers based on the total sum of invoices.

In [59]:
# Getting the data 
cursor.execute("""
    select FirstName, LastName, sum(total) as invoiceTotal
    from Customer c
    inner join Invoice i
        on c.CustomerId = i.CustomerId
    group by 1
    order by 3 desc
    limit 3
""")
rows = cursor.fetchall()

# printing them
rows

[('Frank', 'Harris', 81.24),
 ('Mark', 'Philips', 75.24),
 ('Helena', 'Holý', 49.62)]

In [61]:
# exporting the results to csv
colnames = [d[0] for d in cursor.description]
pd.DataFrame(rows).to_csv('Question3_Task2_Query3.csv', header=colnames, index=False)

## Query 4: Generate a report listing all employees who have sold over a specified amount (provide examples for amounts 1000,1000,5000).

In [55]:
# Getting the data 
cursor.execute("""
    select e.FirstName, e.LastName, sum(total) as invoiceTotal
    from Employee e
    inner join Customer c
        on e.EmployeeId = c.SupportRepId
    inner join Invoice i
        on c.CustomerId = i.CustomerId
    group by 1
    having sum(total) > 800
    order by 3 desc
""")
rows = cursor.fetchall()

# printing them to check
rows

In [57]:
# exporting the results to csv
colnames = [d[0] for d in cursor.description]
pd.DataFrame(rows).to_csv('Question3_Task2_Query4.csv', header=colnames, index=False)

## A few insights
1. North America (USA and Canada) have the most sales and customers. Followed by France and Brazil.
2. The top 3 countries in terms of Sales (USA, Canada and France) represents 43.8% of total Sales.
3. The most added track in playlists is "A Midsummer Night's Dream, Op.61 Incidental Music: No.7 Notturno" from Metallica, present in 5 playlists.
4. There is no track not included in at least one playlist. 

In [89]:
# Getting the data 
cursor.execute(""" 
    select BillingCountry, sum(total), count(distinct c.CustomerId),  avg(total)
    from Invoice i
    inner join Customer c
        on c.CustomerId = i.CustomerId
    group by 1
    order by 3 desc
""")
rows = cursor.fetchall()

# and checking a few rows
rows

[('USA', 523.06, 13, 5.747912087912088),
 ('Canada', 303.96, 8, 5.4278571428571425),
 ('France', 195.1, 5, 5.574285714285714),
 ('Brazil', 190.1, 5, 5.4314285714285715),
 ('Germany', 156.48, 4, 5.588571428571428),
 ('United Kingdom', 112.86, 3, 5.3742857142857146),
 ('Portugal', 77.24, 2, 5.517142857142857),
 ('India', 75.26, 2, 5.78923076923077),
 ('Czech Republic', 90.24, 2, 6.445714285714286),
 ('Sweden', 38.62, 1, 5.517142857142857),
 ('Spain', 37.62, 1, 5.374285714285714),
 ('Poland', 37.62, 1, 5.374285714285714),
 ('Norway', 39.62, 1, 5.659999999999999),
 ('Netherlands', 40.62, 1, 5.8028571428571425),
 ('Italy', 37.62, 1, 5.374285714285714),
 ('Ireland', 45.62, 1, 6.517142857142857),
 ('Hungary', 45.62, 1, 6.517142857142857),
 ('Finland', 41.62, 1, 5.945714285714286),
 ('Denmark', 37.62, 1, 5.374285714285714),
 ('Chile', 46.62, 1, 6.659999999999999),
 ('Belgium', 37.62, 1, 5.374285714285714),
 ('Austria', 42.62, 1, 6.088571428571428),
 ('Australia', 37.62, 1, 5.374285714285714),


In [ ]:
# Getting the data 
cursor.execute(""" 
    select sum(case when BillingCountry in ('USA','Canada','France') then total end) / sum(total)
    from Invoice i
    inner join Customer c
        on c.CustomerId = i.CustomerId
""")
rows = cursor.fetchall()

# and checking a few rows
rows

In [115]:
# Getting the data 
cursor.execute(""" 
    select t.name as trackName, a.Title, ar.name as artistName, count(distinct PlaylistId) as nbPlaylist
    from Track t
    inner join Album a
        on t.AlbumId = t.AlbumId
    inner join Artist ar
        on a.ArtistId = ar.ArtistId
    inner join PlaylistTrack p
        on t.TrackId = p.TrackId
    group by 1,2,3
    order by 4 desc
    limit 1
""")
rows = cursor.fetchall()

# and checking a few rows
rows

[("A Midsummer Night's Dream, Op.61 Incidental Music: No.7 Notturno",
  '...And Justice For All',
  'Metallica',
  5)]

In [127]:
# Getting the data 
cursor.execute(""" 
    select count(TrackId) as nb
    from Track t
    where not exists (select 1 from PlaylistTrack p where t.TrackId = p.TrackId)
""")
rows = cursor.fetchall()

# and checking a few rows
rows

[(0,)]

## Final cleaning

In [67]:
# Committing the changes if needed
conn.commit()
 
# closing the database connection
conn.close()